### Environment Setup
This section imports necessary libraries, loads environment variables, and retrieves API keys required for interacting with external services like Google Vertex AI and Cal.com API.

In [1]:
import os
import requests
import json


from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY= os.environ.get("GOOGLE_API_KEY")
CALCOM_API_KEY=os.environ.get("CALCOM_API_KEY")

### Vertex AI Initialization
Here, we initialize the Google Vertex AI platform with the specified project ID and location. We also load the Gemini generative model, which will be used later for AI-driven tasks

In [2]:
import vertexai
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

PROJECT_ID = "YOUR PROJECT_ID " # @param {type:"string"}
LOCATION = "LOCATION"  # @param {type:"string"}

vertexai.init(project=PROJECT_ID, location=LOCATION)

# Initialize Gemini model gemini-1.0-pro
model = GenerativeModel(model_name="gemini-1.0-pro-001")

### Fetching Bookings
This function retrieves all bookings for a given email address from Cal.com using the API. It handles any potential request errors and returns the response in JSON format.

In [3]:
def get_all_bookings(attendee_email):
    """
    Retrieves all bookings for a given attendee email address using the Cal.com API.

    Args:
        attendee_email (str): The email address of the attendee for whom to retrieve bookings.

    Returns:
        dict: A dictionary containing the parsed JSON response from the API,
              or None if the request failed.

    Raises:
        requests.exceptions.RequestException: If an error occurs during the request.
    """

    url = "https://api.cal.com/v1/bookings"
    query_params = {"apiKey": CALCOM_API_KEY}  # Replace with your actual key
    data = {"attendeeEmail": attendee_email}

    try:
        response = requests.get(url, params=query_params, json=data)
        response.raise_for_status()  # Raise an exception for non-2xx status codes
        return response.json()

    except requests.exceptions.RequestException as e:
        return json.dumps({"error": "An unexpected error occurred", "details": str(e)})


This function checks available time slots for a given date using the Cal.com API. It processes the response, converts the times to the desired timezone, and returns them in a readable format.

In [4]:
from datetime import datetime, timezone, timedelta

def slot_availbility(date):
    
    url = "https://api.cal.com/v1/slots"
    #print(f'{date}T00:00:00.000Z')

    query_params = {
        "apiKey": CALCOM_API_KEY,
        "eventTypeId": 1237037,
        "startTime": f'{date}T00:00:00.000Z',                
        "endTime": f'{date}T23:45:00.000Z' 

    }
    
    
    try:
        response = requests.get(url, params=query_params)
        response.raise_for_status()  # Raise exception for non-200 status codes
        data= response.json()
        times = []
        for date, slots in data['slots'].items():
            for slot in slots:
            # Parse the time and extract only the time portion (HH:MM:SS)
                time_str = slot['time']
                utc_offset = timezone(timedelta(hours=-5))
                time_obj = datetime.fromisoformat(time_str.replace('Z', '+00:00')).astimezone(utc_offset)
                times.append(time_obj.strftime('%H:%M:%S'))
        return times
    except requests.exceptions.RequestException as e:
        raise Exception(f"Error :{e}")

In [5]:
def create_booking(start,email,date):
    """Creates a new booking on a calendar API.

    Args:
      start (str): The start time of the booking in ISO 8601 format (e.g., "2024-10-25T10:00:00").
      end (str): The end time of the booking in ISO 8601 format (e.g., "2024-10-25T10:30:00").
      email (str): The email address of the attendee.
      date(str): Booking date in YYYY-MM-DD FORMAT 
      title (str, optional): The title of the booking. Defaults to "Team Meeting".
      description (str, optional): The description of the booking. Defaults to "Meeting Description".
      status (str, optional): The status of the booking (e.g., "confirmed", "pending"). Defaults to "confirmed".

    Returns:
      dict: The JSON response from the API call, containing information about the created booking.
          Raises an exception if the API call fails.

    Raises:
      Exception: If the API call returns an error code or unexpected response.
    """
    # check availbility
    available_slots=slot_availbility(date)
    
    url = "https://api.cal.com/v1/bookings"

    query_params = {
      "apiKey": CALCOM_API_KEY
    }
    start_time=f'{date}T{start}-05:00'

    if start in available_slots:
        print("start_time",start_time)
    
        data = {
            "eventTypeId": int(1237037),
            "start": start_time,
            "responses": {
              "name": "gayathri",
              "email": email,
              "guests": [],
              "location": {
                  "value": "integrations:calcom",
                  "optionValue": ""
              }
            },
            "metadata": {},
            "timeZone": "America/New_York",  # Adjust if needed
            "language": "en",
            "title": "",
            "description": "",
            "status": "confirmed",
            "seatsPerTimeSlot": 10,
            "seatsShowAttendees": True,
            "seatsShowAvailabilityCount": True
        }
    
        try:
            response = requests.post(url, params=query_params,json=data)
            response.raise_for_status()  # Raise exception for non-200 status codes
            return response.json()
        except requests.exceptions.RequestException as e:
            raise Exception(f"Error creating booking: {e}")
    else:
        return f'slot time is not avaible. Available slot times {available_slots}'

In [6]:
def edit_booking(id, title,description,status="CANCELLED"):
    """Edits a booking on a calendar API.
    Args:
      id (int): The ID of the booking to edit.
      title (str, optional): The new title for the booking. Defaults to "".
      start (str, optional): The new start time for the booking (format depends on API). Defaults to "".
      end (str, optional): The new end time for the booking (format depends on API). Defaults to "".
      status (str, optional): The new status for the booking. Defaults to "CANCELLED".
      
    Returns:
      dict: The JSON response from the API call.
      
      """
    url = f"https://api.cal.com/v1/bookings/{id}"

    query_params = {
      "apiKey": CALCOM_API_KEY  # Replace with your actual API key
    }

    data = {
        "title": title,
        "status": status,
        "description":description
    }

    response = requests.patch(url, params=query_params, json=data)  # Use PUT for editing

    if response.status_code == 200:
      return response.json()
    else:
      raise Exception(f"Error editing booking: {response.text}")

In [7]:
def cancel_booking(booking_id):
    """
    Cancels a booking on a Cal.com calendar using the provided API key and booking ID.

    Args:
        booking_id (int): The unique identifier of the booking to cancel.
        api_key (str): Your Cal.com API key. Replace with your actual key.

    Returns:
        dict: A dictionary containing the response data from the API call,
             or None if the request failed.

    Raises:
        requests.exceptions.RequestException: If an error occurs during the HTTP request.
    """

    url = f"https://api.cal.com/v1/bookings/{booking_id}/cancel"
    query_params = {
      "apiKey": CALCOM_API_KEY  # Replace with your actual API key
    }

    try:
        response = requests.delete(url, params=query_params)
        response.raise_for_status()  # Raise an exception for non-2xx status codes

        if response.content:
            return response.json()  # Return JSON data if present
        else:
            return {}  # Return empty dict if response has no content

    except requests.exceptions.RequestException as e:
        return json.dumps({"error": "An unexpected error occurred", "details": str(e)})


In [8]:
def run(user_query):

    # function declarations 
    get_all_bookings_func = FunctionDeclaration(
        name="get_all_bookings",
        description="Retrieves all bookings for a given attendee email address.",
        parameters={
            "type": "object",
            "properties": {"attendee_email": {"type": "string", "description": "email of the attendee"}},
        },
    )
    
    
    create_booking_func = FunctionDeclaration(
        name="create_booking",
        description="Creates a new booking with specified event type, start, end, and attendee email.",
        parameters={
            "type": "object",
            "properties": {
                "start": {"type": "string", "description": "start time for the booking ISO 8601 format"},
                "date":{"type":"string", "description": "date in YYYY-MM-DD format, consider year as 2024 if not provided"},
                "email": {"type": "string", "description": "Attendee email"},
            },
            "required": ["start","email","date"]
        },
    )
    
    edit_booking_func = FunctionDeclaration(
        name="edit_booking",
        description="Edits an existing booking with specified ID, start, end, and attendee email.",
        parameters={
            "type": "object",
            "properties": {
                "booking_id": {"type": "integer", "description": "ID of the booking to edit"},
                "title":{"type": "string", "description": "new title of the booking"},
                "start": {"type": "string", "description": "new start time for the booking ISO 8601 format with a -5 timezone offset"},
                "end": {"type": "string", "description": "new End time for the booking ISO 8601 format with a -5 timezone offset"},
                "description": {"type": "string", "description": "#new status for the booking. Allowed values('CANCELLED' | 'ACCEPTED' | 'REJECTED' | 'PENDING' | 'AWAITING_HOST')"},
            },
            "required": ["booking_id"]
        },
    )
    
    
    cancel_booking_func = FunctionDeclaration(
        name="cancel_booking",
        description="Cancels an existing booking with specified ID.",
        parameters={
            "type": "object",
            "properties": {"booking_id": {"type": "integer", "description": "ID of the booking to cancel"}},
            "required": ["booking_id"]
        },
    )
    
    booking_tool = Tool(
        function_declarations=[
            get_all_bookings_func,
            create_booking_func, 
            edit_booking_func, 
            cancel_booking_func,
        ],
    )
    
    model = GenerativeModel(
        model_name="gemini-1.0-pro",
        generation_config=GenerationConfig(temperature=0),
        tools=[booking_tool],
    )

    # Start a chat session
    chat = model.start_chat()
    
    
    function_response = chat.send_message(user_query)

    
    try:
        function_call=function_response.candidates[0].function_calls[0].name
        function_args=function_response.candidates[0].function_calls[0].args


        if function_call=='get_all_bookings':
            attendee_email=function_args['attendee_email']
            api_response=get_all_bookings(attendee_email)
            
        elif function_call=="create_booking":
            start=function_args['start']
            email=function_args['email']
            date=function_args['date']
            
            api_response=create_booking(start,email,date)
    
        elif function_call=="edit_booking":
            booking_id=function_args['booking_id']
            title = function_args['title'] if 'title' in function_args else " "
            description = function_args['description'] if 'description' in function_args else " "
            
            api_response=edit_booking(booking_id,title,description)
    
        elif function_call=="cancel_booking":
            booking_id=function_args['booking_id']
            api_response=cancel_booking(booking_id)

        response = chat.send_message(
            Part.from_function_response(
                name=function_call,
                response={
                    "content": api_response,
                },
            ),
        )   
        return response.text,function_response
        
    except:
    # No function call, handle the regular response
        model_response = function_response.candidates[0].text
        print(model_response)
        return model_response,function_response




In [9]:
response,function_response=run("What are gayathrip556@gmail.com's scheduled meetings?")
print(response.text)

## Gayathri's Scheduled Meetings:

**1. Cancelled Meeting:**
* **Date:** 2024-09-24
* **Time:** 2:00 PM - 2:30 PM EST
* **Title:** a meeting between gayathri and her collegue
* **Description:** 

**2. Accepted Meeting:**
* **Date:** 2024-09-24
* **Time:** 3:45 PM - 4:00 PM EST
* **Title:** 15 Min Meeting between Gayathri Pittu and Gayathri
* **Description:** 

**3. Cancelled Meeting:**
* **Date:** 2024-09-26
* **Time:** 6:00 PM - 6:30 PM EST
* **Title:** 30 Min Meeting between Gayathri Pittu and gayathrip
* **Description:** 

**4. Accepted Meeting:**
* **Date:** 2024-10-09
* **Time:** 5:00 PM - 5:15 PM EST
* **Title:** 15min between Gayathri Pittu and gayathri
* **Description:** 

**5. Accepted Meeting:**
* **Date:** 2024-10-08
* **Time:** 3:00 PM - 3:15 PM EST
* **Title:** 15min between Gayathri Pittu and gayathri
* **Description:** 

**6. Accepted Meeting:**
* **Date:** 2024-10-15
* **Time:** 3:00 PM - 3:30 PM EST
* **Title:** 30min between Gayathri Pittu and John Doe
* **Description

This cell provides insights into the program's internal logic. It explains how the Large Language Model (LLM) processed the query and determined the appropriate function to use. The LLM identified the "get_all_bookings" function and supplied the necessary arguments, including the attendee email address ("gayathrip556@gmail.com") used to filter the retrieved bookings.

In [10]:
function_response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_all_bookings"
        args {
          fields {
            key: "attendee_email"
            value {
              string_value: "gayathrip556@gmail.com"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.265625
    severity: HARM_SEVERITY_LOW
    severity_score: 0.202148438
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.345703125
    severity: HARM_SEVERITY_LOW
    severity_score: 0.341796875
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.267578125
    severity: HARM_SEVERITY_LOW
    severity_score: 0.255859375
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    prob

In [11]:
response,function_response=run("Change my meeting with 3613259 title to 'meeting on project updates.'")
print(response.text)

OK. I've changed the title of your meeting with ID 3613259 to "meeting on project updates.". Is there anything else?


In [12]:
function_response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "edit_booking"
        args {
          fields {
            key: "title"
            value {
              string_value: "meeting on project updates."
            }
          }
          fields {
            key: "booking_id"
            value {
              number_value: 3613259
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.138671875
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.138671875
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.298828125
    severity: HARM_SEVERITY_LOW
    severity_score: 0.211914062
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.1875
    severity: HARM_SEVERITY_NEGLIGIBLE
  

In [15]:
response,function_response=run("Remove my meeting with 3412524")
print(response.text)

OK. I have cancelled the booking with ID 3412524. 
Is there anything else?


In [16]:
function_response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "cancel_booking"
        args {
          fields {
            key: "booking_id"
            value {
              number_value: 3412524
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.198242188
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.151367188
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.5
    severity: HARM_SEVERITY_LOW
    severity_score: 0.357421875
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.32421875
    severity: HARM_SEVERITY_LOW
    severity_score: 0.200195312
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.13378

In [17]:
response,function_response=run("create a booking on october 14th  at 11.30am with Olive@calc.com")
print(response.text)

start_time 2024-10-14T11:30:00-05:00
OK. I've created a booking for Olive@calc.com on October 14th at 11:30am. 

Here are the details:

* **Date:** 2024-10-14
* **Time:** 11:30am
* **Email:** Olive@calc.com
* **Title:** 15min between Gayathri Pittu and gayathri
* **Status:** ACCEPTED

Is there anything else I can help you with?


In [46]:
response,function_response=run("create a booking on tuesday")
print(response)

What is your email? 
What time would you like to book it for?
What is your email? 
What time would you like to book it for?


In [47]:
function_response

candidates {
  content {
    role: "model"
    parts {
      text: "What is your email? \nWhat time would you like to book it for?"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.06640625
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0693359375
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.1484375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.117675781
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.21875
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0583496094
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.166015625
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.134765625
  }
  avg_logprobs: -0.13730214623843923
}
usa

In [48]:
response,function_response=run("give me available slots")
response

What is the attendee's email?


"What is the attendee's email?"

In [49]:
function_response

candidates {
  content {
    role: "model"
    parts {
      text: "What is the attendee\'s email?"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.101074219
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.103515625
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.271484375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.078125
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.184570312
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0727539062
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.102539062
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0717773438
  }
  avg_logprobs: -0.3038458526134491
}
usage_metadata {
  prompt_token_